<a href="https://colab.research.google.com/github/jsandtangen/Book-Recommendation/blob/main/Book_Recommendation_Engine_using_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2026-01-07 09:05:05--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   142MB/s    in 0.2s    

2026-01-07 09:05:05 (142 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [7]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [6]:
df_ratings.tail()

,user,isbn,rating
1149775,276704,1563526298,9.0
1149776,276706,0679447156,0.0
1149777,276709,0515107662,10.0
1149778,276721,0590442449,10.0
1149779,276723,05162443314,8.0


In [8]:
user_counts = df_ratings['user'].value_counts()
isbn_counts = df_ratings['isbn'].value_counts()

df = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]

df = df[df['isbn'].isin(isbn_counts[isbn_counts >= 100].index)]

df = df.merge(df_books, on='isbn', how='inner')
df = df.drop_duplicates(['title', 'user'])

In [9]:
piv = df.pivot(index='title', columns='user', values='rating').fillna(0)

matrix = piv.values

In [10]:
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [11]:
# function to return recommended books - this will be tested
def get_recommends(book=""):
    # Sjekk om boken finnes i datasettet
    if book not in piv.index:
        raise ValueError("Boken finnes ikke i datasettet")

    # Hent rad-vektoren for boken (alle brukernes ratinger)
    bok_vektor = piv.loc[book].values.reshape(1, -1)

    # Finn de 6 nærmeste naboene (mest like bøker)
    distances, indices = model.kneighbors(bok_vektor, n_neighbors=6)

    # Lag en liste for anbefalingene
    anbefalinger = []

    # Gå gjennom avstandene og indeksene til naboene
    for avstand, radnummer in zip(distances[0], indices[0]):
        # Hopp over boken selv (den har avstand 0)
        if avstand != 0:
            tittel = piv.index[radnummer]
            anbefalinger.append([tittel, round(float(avstand), 2)])

    # Snu rekkefølgen slik at mest lik kommer sist (lavest avstand)
    anbefalinger = anbefalinger[::-1]

    # Returner original bok + anbefalingene
    return [book, anbefalinger]

In [12]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8], ['The Weight of Water', 0.77], ['The Surgeon', 0.77], ['I Know This Much Is True', 0.77], ['The Lovely Bones: A Novel', 0.72]]]
You passed the challenge! 🎉🎉🎉🎉🎉
